<a href="https://colab.research.google.com/github/basan4ik/similarity-search/blob/main/Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libs, files and short EDA

Let's download necessary libraries

In [3]:
#!pip install faiss-gpu
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 24.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import faiss

Mount drive disk where our files located

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/similarity-search/data

base.csv  train.csv  validation_answer.csv  validation.csv


Open file base.csv as Pandas dataframe

In [8]:
with open('/content/drive/MyDrive/similarity-search/data/validation.csv', 'r') as f:
  validation = pd.read_csv(f, index_col=0)

In [9]:
validation.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


In [15]:
validation.shape

(100000, 72)

In [12]:
with open('/content/drive/MyDrive/similarity-search/data/validation_answer.csv', 'r') as f:
  validation_answer = pd.read_csv(f, index_col=0)

In [13]:
validation_answer.head()

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


In [16]:
validation_answer.shape

(100000, 1)

In [ ]:
with open('/content/drive/MyDrive/similarity-search/data/base.csv', 'r') as f:
  base = pd.read_csv(f, index_col=0)

In [ ]:
base.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [ ]:
base.shape

(2918139, 72)

In [ ]:
base.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,...,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,-8.622947e+01,8.080077e+00,-4.458080e+01,-1.466350e+02,1.113166e+02,-7.199138e+01,-3.922239e+02,2.035283e+01,1.236842e+02,1.244581e+02,...,-7.902286e+01,3.329735e+01,-1.547962e+02,1.415132e+01,6.779167e+01,2.354490e+01,7.495930e+01,1.155667e+02,-7.993390e+02,-4.779125e+01
std,2.489132e+01,4.953387e+00,3.863166e+01,1.984480e+01,4.634809e+01,2.818607e+01,2.716550e+02,6.421638e+01,6.356109e+00,6.443058e+01,...,3.045642e+01,2.888603e+01,4.122929e+01,9.895115e+01,1.823356e+00,5.534224e+01,6.134500e+01,2.117518e+01,3.854131e+02,4.174802e+01
min,-1.994687e+02,-1.391461e+01,-2.400734e+02,-2.326671e+02,-1.055830e+02,-2.110086e+02,-7.914699e+02,-3.018597e+02,9.315305e+01,-1.738719e+02,...,-2.205662e+02,-8.850774e+01,-3.539028e+02,-1.575944e+02,5.950944e+01,-2.331382e+02,-2.036016e+02,1.572448e+01,-1.297931e+03,-2.267801e+02
25%,-1.030654e+02,4.708491e+00,-6.955949e+01,-1.599051e+02,8.050795e+01,-9.137994e+01,-6.293318e+02,-2.222147e+01,1.194840e+02,8.176751e+01,...,-9.876390e+01,1.698862e+01,-1.807799e+02,-7.130038e+01,6.658096e+01,-1.251624e+01,3.377574e+01,1.016867e+02,-1.074465e+03,-7.566641e+01
50%,-8.623150e+01,8.038950e+00,-4.381661e+01,-1.467768e+02,1.118730e+02,-7.192230e+01,-4.222016e+02,2.080477e+01,1.238923e+02,1.234977e+02,...,-7.848812e+01,3.471502e+01,-1.539773e+02,1.382693e+01,6.781458e+01,2.341649e+01,7.492997e+01,1.160244e+02,-1.074465e+03,-4.859196e+01
75%,-6.925658e+01,1.147007e+01,-1.962527e+01,-1.333277e+02,1.423743e+02,-5.244111e+01,-1.566686e+02,6.391821e+01,1.279705e+02,1.672206e+02,...,-5.853355e+01,5.216429e+01,-1.273405e+02,9.966753e+01,6.902666e+01,5.975511e+01,1.158760e+02,1.295524e+02,-5.057445e+02,-1.971424e+01
max,2.151555e+01,2.993721e+01,1.609372e+02,-5.137478e+01,3.196645e+02,5.880624e+01,1.096325e+02,3.412282e+02,1.522612e+02,4.275421e+02,...,6.017411e+01,1.541678e+02,2.436099e+01,1.850981e+02,7.571203e+01,3.148988e+02,3.395738e+02,2.147063e+02,9.877081e+01,1.269732e+02


In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2918139 entries, 0-base to 4744766-base
Data columns (total 72 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64


In [ ]:
base = base.astype('float32')

Let's make sure we don't have any missing values

In [ ]:
pd.set_option('display.max_rows', None)
base.isna().sum()

As we can see we have a dataframe with 2918139 rows with 72 float value columns. It has no missing values.

From now on we assume that 'base.csv' file is full of ready to use embeddings (vector representations of pictures, texts or something else).

In [6]:
with open('/content/drive/MyDrive/similarity-search/data/train.csv', 'r') as f:
  train = pd.read_csv(f, index_col=0)

In [7]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


In [ ]:
train.shape

(100000, 73)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21  

In [ ]:
train_target = train['Target']
train_features = train.drop(['Target'], axis=1)
train_target.head()

Id
0-query     675816-base
1-query     366656-base
2-query    1447819-base
3-query    1472602-base
4-query     717819-base
Name: Target, dtype: object

In [ ]:
train_features = train_features.astype('float32')

## Research
### Building an index and adding the vectors to it

We need two matrices:

- xb for the database, that contains all the vectors that must be indexed, and that we are going to search in. Its size is nb-by-d
- xq for the query vectors, for which we need to find the nearest neighbors. Its size is nq-by-d. If we have a single query vector, nq=1.

In [ ]:
d = 72                           # dimension
nb = 2918139                      # database size
nq = 100000                       # nb of queries
np.random.seed(1234)             # make reproducible

In [ ]:
res = faiss.StandardGpuResources()  # use a single GPU
# make it a flat GPU index
index_flat = faiss.IndexFlatL2(d)   # build the index
print(index_flat.is_trained)

True


In [ ]:
# make it a flat GPU index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

In [ ]:
gpu_index_flat.add(np.ascontiguousarray(base.values))         # add vectors to the index
print(gpu_index_flat.ntotal)

2918139


### Searching

In [ ]:
k = 5                          # we want to see 5 nearest neighbors
D, I = gpu_index_flat.search(np.ascontiguousarray(train_features.values), k)  # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[1480698  161948 1076334 1882633 1282393]
 [ 445586  920175 2168908 2651198  546230]
 [1659033  760940  656828 1052397 1392119]
 [2825385 1573375 2745252  684927  429085]
 [ 212436 1304565 1332011 1338879  595387]]
[[2367775  163010 1872168 1569400   52963]
 [1898264 2294134 1139084 1870027  514241]
 [2690258  700937 1242425 2703195  813545]
 [1197717 2500439 2265191 2358202  955468]
 [1958647  272735  458565  231187 1863959]]


The code below is copied from this notebook https://colab.research.google.com/drive/1WUG6JO6ra4W3bs6Wi7febuPrd9B4D61B#scrollTo=d42af9e0-3f09-4b4c-a1a6-688387db17de

In [ ]:
base_index = {k: v for k, v in enumerate(base.index.to_list())}

In [ ]:
acc = 0
for target, el in zip(train_target.values.tolist(), I.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(I))

13.804


We got accuracy 13.804

Ok.

In [ ]:
type(base_index)

dict

In [17]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [18]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [19]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751738 7.20763   7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279875 7.662846 ]
 [0.        6.7638035 7.2951202 7.3688145]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [23]:
print(D[:5])

[[6.815506  6.8894653 7.3956795 7.4290257]
 [6.6041145 6.679699  6.7209625 6.828678 ]
 [6.4703865 6.8578644 7.0043755 7.036564 ]
 [5.573681  6.4075394 7.1395187 7.3555946]
 [5.409401  6.232216  6.4173393 6.5743675]]


In [20]:
I.shape

(10000, 4)

In [21]:
xb[:5]

array([[0.19151945, 0.62210876, 0.43772775, 0.7853586 , 0.77997583,
        0.2725926 , 0.27646425, 0.8018722 , 0.95813936, 0.87593263,
        0.35781726, 0.5009951 , 0.6834629 , 0.71270204, 0.37025076,
        0.5611962 , 0.50308317, 0.01376845, 0.7728266 , 0.8826412 ,
        0.364886  , 0.6153962 , 0.07538124, 0.368824  , 0.9331401 ,
        0.65137815, 0.39720258, 0.78873014, 0.31683612, 0.56809866,
        0.8691274 , 0.4361734 , 0.8021476 , 0.14376682, 0.70426095,
        0.7045813 , 0.21879211, 0.92486763, 0.44214076, 0.90931594,
        0.05980922, 0.18428709, 0.04735528, 0.6748809 , 0.59462476,
        0.5333102 , 0.04332406, 0.5614331 , 0.32966843, 0.5029668 ,
        0.11189432, 0.6071937 , 0.5659447 , 0.00676406, 0.6174417 ,
        0.9121229 , 0.7905241 , 0.99208146, 0.95880175, 0.7919641 ,
        0.28525096, 0.62491673, 0.4780938 , 0.19567518],
       [0.38331744, 0.05387368, 0.4516484 , 0.98200476, 0.1239427 ,
        0.1193809 , 0.73852307, 0.58730364, 0.47163254, 0.1

In [ ]:
arr = np.random.random((nb, 1)).astype('float32')
for target, el in zip(arr.tolist(), xb.tolist()):
  print(target, el)
  print('Next')

[0.3330024778842926] [0.19151945412158966, 0.6221087574958801, 0.43772774934768677, 0.7853586077690125, 0.7799758315086365]
Next
[0.7284286618232727] [0.27359259128570557, 0.2764642536640167, 0.801872193813324, 0.9581393599510193, 0.8759326338768005]
Next
[0.14243537187576294] [0.3598172664642334, 0.5009950995445251, 0.683462917804718, 0.7127020359039307, 0.37025076150894165]
Next
[0.5524689555168152] [0.5641962289810181, 0.5030831694602966, 0.013768449425697327, 0.772826611995697, 0.8826411962509155]
Next
[0.2730432450771332] [0.36888599395751953, 0.6153962016105652, 0.07538124173879623, 0.3688240051269531, 0.9331400990486145]
Next
[0.9744951128959656] [0.6563781499862671, 0.39720258116722107, 0.7887301445007324, 0.3168361186981201, 0.5680986642837524]
Next
[0.6677868962287903] [0.875127375125885, 0.4361734092235565, 0.802147626876831, 0.14376682043075562, 0.7042609453201294]
Next
[0.2556532919406891] [0.7115813493728638, 0.2187921106815338, 0.9248676300048828, 0.44214075803756714, 0.

In [ ]:
type(xb)

numpy.ndarray